# <center>NLP: Множественная классификация текстов отзывов.</center>
---

## Решение 2.

Т.к. точно размеченных данных относительно немного, проведём эксперимент с самообучением модели с псевдоразметкой: обучение классификатора на размеченных данных и последующее итеративное включение предсказаний с заданной точностью в обучающую выборку с переобучением модели.

### План работы.
- Модели:
  - `CatBoostClassifier`.
- Данные:
  - создадим эмбеддинги текста с помощью более легковесной `sentence-transformers/paraphrase-multilingual-mpnet-base-v2`
  - тэги закодируем как бинарные признаки;
  - оценки оставим без изменений.
- Результат:<br>
Решение не сработало. Хорошим экспериментом была бы проверка самообучения большой языковой модели типа `ruGPT` или подобных, на работу с которыми требуется больше времени и ресурсов.<br>

### Данные
- `train.csv.zip` - обучающая выборка;

|Название столбца|Описание столбца|
|:-|:-|
|index|Id объекта|
|assessment|Числовая оценка заказа(ов)|
|tags|Тематические теги|
|text|Текстовый коментарий|
|trend_id_res$X$|Код класса: $X ∈ [1, 50]$|

- `test.csv.zip` - тестовая выборка: аналогична обучающей, но без классов;
- `sample_submission.csv.zip` - пример посылки.

## Загрузка библиотек.
---

In [12]:
## os для обращения к среде разработки.
import os, sys

## Работа с датафреймами и датасетами.
import pandas as pd
## Визуализация.
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns

## numpy для вычислений.
import numpy as np, random

## Модели и модули.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# # CatBoost классификатор.
!pip install catboost -q
from catboost import CatBoostClassifier, Pool
# Получение предобученных моделей и модулей.
!pip install sentence_transformers -q
from sentence_transformers import SentenceTransformer

## Дополнительные модули.
# tqdm для контроля выполнения ячеек.
from tqdm.notebook import tqdm, trange
# Аннотация типов.
from typing import Optional, List, Union
# Подавление предупреждений.
import warnings

## Установка параметров ноутбука.
---

### Установка пути к данным.

In [2]:
# Зададим путь и название датасета.
PATH = None # Задайте путь к данным
SAVE_PATH = None # Задайте путь сохранения данных
EMBEDDINGS_PATH = 'embeddings/self_train/'
TRAIN = 'train.csv.zip'
TEST = 'test.csv.zip'

if not PATH:
    if 'google.colab' in sys.modules:
        env = 'colab'
        from google.colab import drive
        drive.mount('/content/drive')
        PATH = ('/content/drive/Othercomputers/PC/JupiterNB/Yandex_Practicum_'+
                'Workshop/NLP_Multilabel_Samokat/__repo/Notebooks/Data'+
                '/dataset/')
    elif 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
        env = 'kaggle'
        PATH = '/kaggle/input/ya-workshop-samokat-2025/'
        TRAIN = 'train.csv'
        TEST = 'test.csv'
        TRENDS = 'trends_description.csv'
    elif 'ipykernel' in sys.modules:
        env = 'local'
        PATH = '../Data/dataset/'

if not SAVE_PATH:
    if env == 'kaggle':
        SAVE_PATH = '/kaggle/working/'
    elif env == 'colab':
        SAVE_PATH = ('/content/drive/Othercomputers/PC/JupiterNB/Yandex_'+
                     'Practicum_Workshop/NLP_Multilabel_Samokat/'+
                     '__repo/Notebooks/Data/')
    elif env == 'local':
        SAVE_PATH = '../Data/'

### Установка других параметров.

In [3]:
# Подавление предупреждений.
warnings.filterwarnings('ignore')

# Установим начальное значение генератора псевдослучайных чисел.
SEED = 42

def set_rnd_state(seed : int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f'Случайные числа инициализированы с начальным значением {seed}.')

set_rnd_state(SEED)

# Функция вывода семпла датафрейма в стиле.
def display_s(df : pd.DataFrame | pd.Series,
              name : str = 'Датафрейм') -> None:
    if type(df) == pd.Series:
        df = df.to_frame()
    if df.shape[1] <= 3:
        df = df.T
    display(df.style.set_caption(name).set_table_styles(styles))

# Установим параметры отображения таблиц.
for opt in ['max_columns', 'max_rows', 'min_rows']:
    pd.set_option(f'display.{opt}', 1000)
pd.set_option('display.max_colwidth', None)

# Установим стиль отображения заголовков таблиц.
styles = [
    dict(selector = 'caption',
         props = [('text-align', 'center'),
                  ('font-size', '130%'),
                  ('font-weight', 'bold'),
                  ('white-space', 'pre-wrap')]),
    dict(selector = 'th',
         props = [('max-width', '250px')])
    ]

# Установим параметры отображения графиков.
MPL_WIDTH = 15
MPL_HEIGHT = 5
rc('font', size = 14)
figure_kwargs = dict(figsize = (MPL_WIDTH, MPL_HEIGHT),
                     facecolor = 'white',
                     autolayout = True,
                     titlesize = 16)
plt.style.use('seaborn-v0_8-whitegrid')
rc('figure', **figure_kwargs)

Случайные числа инициализированы с начальным значением 42.


## Загрузка данных.
---

In [4]:
data = pd.read_csv(os.path.join(PATH, TRAIN))
test_unk = pd.read_csv(os.path.join(PATH, TEST))

## Предобработка.



- Удалим объекты с пропущенными категориями и оценками.

In [5]:
data.dropna(inplace=True)
print(f'Пропусков после удаления: {data.isna().sum().sum()}')

Пропусков после удаления: 0


- Создадим бинарные признаки из тэгов, содердашихся в признаке `tags` .

In [6]:
def make_binaries_from_multicat_col(data : pd.DataFrame = None,
                                    col : str = None) -> pd.DataFrame:
    data['_list'] = (data[col]
                     .str.replace(r'{|}', '', regex=True)
                     .str.split(','))
    tags_unique = set(data['_list'].dropna().explode().tolist())
    data.drop('_list', axis=1, inplace=True)

    data[[x for x in tags_unique]] = np.zeros(
        (data.shape[0], len(tags_unique))
        )

    for tag in tags_unique:
        data[tag] = data[tag].where(~(data['tags'].str.contains(tag)), 1)

    data.columns = [x.lower() for x in data.columns]

    return data

In [7]:
data = make_binaries_from_multicat_col(data, 'tags')
display(data.head())
print(data.columns)

,index,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,trend_id_res5,trend_id_res6,trend_id_res7,trend_id_res8,trend_id_res9,trend_id_res10,trend_id_res11,trend_id_res12,trend_id_res13,trend_id_res14,trend_id_res15,trend_id_res16,trend_id_res17,trend_id_res18,trend_id_res19,trend_id_res20,trend_id_res21,trend_id_res22,trend_id_res23,trend_id_res24,trend_id_res25,trend_id_res26,trend_id_res27,trend_id_res28,trend_id_res29,trend_id_res30,trend_id_res31,trend_id_res32,trend_id_res33,trend_id_res34,trend_id_res35,trend_id_res36,trend_id_res37,trend_id_res38,trend_id_res39,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49,price,promotions,products_quality,support,payment,assortment,delivery,catalog_navigation
0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортимент больше, а так вроде бы все хорошо",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,15237,5.0,"{ASSORTMENT,PRODUCTS_QUALITY,PROMOTIONS,CATALOG_NAVIGATION}",Доставка просто 👍,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0


Index(['index', 'assessment', 'tags', 'text', 'trend_id_res0', 'trend_id_res1',
       'trend_id_res2', 'trend_id_res3', 'trend_id_res4', 'trend_id_res5',
       'trend_id_res6', 'trend_id_res7', 'trend_id_res8', 'trend_id_res9',
       'trend_id_res10', 'trend_id_res11', 'trend_id_res12', 'trend_id_res13',
       'trend_id_res14', 'trend_id_res15', 'trend_id_res16', 'trend_id_res17',
       'trend_id_res18', 'trend_id_res19', 'trend_id_res20', 'trend_id_res21',
       'trend_id_res22', 'trend_id_res23', 'trend_id_res24', 'trend_id_res25',
       'trend_id_res26', 'trend_id_res27', 'trend_id_res28', 'trend_id_res29',
       'trend_id_res30', 'trend_id_res31', 'trend_id_res32', 'trend_id_res33',
       'trend_id_res34', 'trend_id_res35', 'trend_id_res36', 'trend_id_res37',
       'trend_id_res38', 'trend_id_res39', 'trend_id_res40', 'trend_id_res41',
       'trend_id_res42', 'trend_id_res43', 'trend_id_res44', 'trend_id_res45',
       'trend_id_res46', 'trend_id_res47', 'trend_id_res48'

Создадим списки с названиями признаков.

In [8]:
x_cols = ['text', 'assessment', 'support', 'products_quality', 'price',
          'catalog_navigation','delivery', 'payment', 'promotions', 'assortment']
x_cols.sort()
y_cols = [f'trend_id_res{i}' for i in range(0, 50)]

- Удалим признаки более не требующиеся.
- Создадим копию размеченных данных и выделим "неразмеченные".

In [9]:
data.drop(['index', 'tags'], axis=1, inplace=True)

first_train = data[(data[y_cols] != 0).any(axis=1)].copy()

self_train = data[(data[y_cols] == 0).all(axis=1)].copy()
self_train.reset_index(drop=True, inplace=True)

## Разделение выборок.

- Разделим размеченные данные на обучающую, валидационную и тестовую отложенную выборку.
- Отсортируем признаки во всех длатафреймах.

### Функция разделения выборок.

In [10]:
### Функция разделения на выборки с использованием train_test_split.
def tvt_split(data : pd.DataFrame,
              target : str,
              val_size : float,
              test_size: float,
              show_results : bool = True):

    # Запишем признаки в переменные.
    X = data.drop(target, axis=1)
    y = data[target]

    # Разделение с валидационной выборкой.
    X_t, X_val, y_t, y_val = train_test_split(
        X, y,
        test_size=val_size,
        random_state=SEED
        )

    X_train, X_test, y_train, y_test = train_test_split(
        X_t, y_t,
        test_size=(1/(1-val_size)*test_size),
        random_state=SEED
        )

    # Сбросим индексы выборок.
    for sample in [X_train, y_train, X_val, y_val, X_test, y_test]:
        sample.reset_index(drop=True, inplace=True)

    if show_results:
        # Запись данных и индекса для таблицы проверки.
        size_check = [data.drop(target, axis=1), X_train, X_val, X_test]
        balance_check = [data[target], y_train, y_val, y_test]
        index = ['Исходный датафрейм:', 'Обучающая выборка:',
                    'Валидационная выборка:', 'Тестовая выборка:']

    if show_results:
        ## Выведем таблицу проверки размерностей.
        balance_list = []
        for i in range(len(balance_check)):
            balance_list.append(
                [size_check[i].shape[0],
                f'{(size_check[i].shape[0] / size_check[0].shape[0]):.1%}',
                size_check[i].shape[1]]
            )

        cols = pd.MultiIndex.from_tuples([
            ('Кол-во объектов.', ''),
            ('Процент объектов.', ''),
            ('Кол-во признаков.', ''),
            ])

        display(
            pd.DataFrame(balance_list, columns=cols, index=index)
            .style
            .set_caption('Результат разделения выборок.')
            .set_table_styles(styles)
        )

    ## Выведем результирующие выборки.
    return (X_train, X_val, X_test, y_train, y_val, y_test)

### Разделение выборок.


In [11]:
X_train, X_val, X_test, y_train, y_val, y_test = tvt_split(
    data = first_train,
    target = y_cols,
    val_size = .2,
    test_size = .2
)

X_unlabeled = self_train[x_cols]

X_train = X_train.reindex(sorted(X_train.columns), axis=1)
X_val = X_val.reindex(sorted(X_val.columns), axis=1)
X_test = X_test.reindex(sorted(X_test.columns), axis=1)
X_unlabeled = X_unlabeled.reindex(sorted(X_unlabeled.columns), axis=1)

,Кол-во объектов.,Процент объектов.,Кол-во признаков.
,,,
Исходный датафрейм:,4612,100.0%,10
Обучающая выборка:,2766,60.0%,10
Валидационная выборка:,923,20.0%,10
Тестовая выборка:,923,20.0%,10


## Генерация эмбеддингов.

Создадим/загрузим эмбеддинги текстовых отзывов.
- создадим/загрузим эмбеддинги;
- присоединим их к остальным признакам;
- удалим оригинальный текст.

### Функции создания/загрузки эмбеддингов.

In [12]:
def generate_embeddings(
        df : pd.DataFrame,
        model : any,
        text_name : str,
        file_name : str,
        save_path : str,
        batch_size : int = 400,
        ) -> pd.DataFrame:
    
    embeddings = []
    print('Генерация эмбеддингов.')
    text = df[text_name]
    for i in trange(0, len(text), batch_size):
        batch = text[i:i + batch_size].tolist()
        batch_embeddings = model.encode(batch,
                                        show_progress_bar=False)
        embeddings.extend(batch_embeddings)

    np.save(
        os.path.join(save_path, (f'{file_name}_embeddings.npy')), 
        embeddings
        )
    df = df.join(pd.DataFrame(embeddings), how='left')

    return df

def load_embeddings(
        df : pd.DataFrame, 
        name : str, 
        load_path : str = os.path.join(SAVE_PATH, 'embeddings/')
        ) -> pd.DataFrame:
    
    embeds = np.load(os.path.join(load_path, f'{name}_embeddings.npy'))
    df = df.join(pd.DataFrame(embeds), how='left')
    print(f'Эмбеддинги {name}_embeddings.npy загружены.')

    return df

### Создание/загрузка эмбеддингов.

In [13]:
path_embeds = os.path.join(SAVE_PATH, EMBEDDINGS_PATH)

try:
    X_train = load_embeddings(X_train, 'X_train', path_embeds)
    X_val = load_embeddings(X_val, 'X_val', path_embeds)
    X_test = load_embeddings(X_test, 'X_test', path_embeds)
    X_unlabeled = load_embeddings(X_unlabeled, 'X_unlabeled', path_embeds)
except:
    model = SentenceTransformer('sentence-transformers/'+
                                'paraphrase-multilingual-mpnet-base-v2')

    X_train = generate_embeddings(X_train, model, 'text', 
                                  'X_train', path_embeds)
    X_val = generate_embeddings(X_val, model,'text', 
                                'X_val', path_embeds)
    X_test = generate_embeddings(X_test, model, 'text', 
                                 'X_test', path_embeds)
    X_unlabeled = generate_embeddings(X_unlabeled, model, 'text', 
                                      'X_unlabeled', path_embeds)
    
X_train.drop(['text'], axis=1, inplace=True)
X_val.drop(['text'], axis=1, inplace=True)
X_test.drop(['text'], axis=1, inplace=True)
X_unlabeled.drop(['text'], axis=1, inplace=True)

x_cols.remove('text')
x_cols.extend([i for i in range(0, 768)])

Эмбеддинги X_train_embeddings.npy загружены.
Эмбеддинги X_val_embeddings.npy загружены.
Эмбеддинги X_test_embeddings.npy загружены.
Эмбеддинги X_unlabeled_embeddings.npy загружены.


In [14]:
for df in [X_train, X_val, X_test, X_unlabeled]:
    display(df.head(1))

,assessment,assortment,catalog_navigation,delivery,payment,price,products_quality,promotions,support,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.022274,0.065933,-0.012597,-0.04871,0.005162,-0.009175,0.03628,0.047924,0.153434,0.019028,-0.023127,0.049428,-0.071951,-0.017196,-0.145357,0.028804,0.001755,0.100376,0.117809,0.035495,-0.080133,-0.039135,0.074828,0.009384,-0.095313,-0.18287,-0.010233,0.058422,0.093811,-0.091634,0.150757,-0.013956,-0.100727,-0.084531,0.078462,-0.023036,0.057132,0.089819,-0.026218,0.088319,-0.085693,-0.057213,0.030384,0.113966,-0.154441,-0.085688,0.034779,-0.05102,0.008575,0.044052,0.0186,0.046356,-0.173134,0.029262,0.171334,0.090498,-0.144546,-0.146972,0.008775,0.029669,-0.049863,-0.02183,-0.058985,0.042649,-0.190677,0.079578,0.011314,-0.171323,0.087523,0.077924,-0.076837,0.017929,0.006267,0.151907,-0.180661,-0.134625,0.029569,-0.021355,0.033527,-0.030386,-0.034191,-0.096944,-0.011079,0.06771,0.088359,0.358647,-0.016088,-0.075308,-0.011706,-0.061667,-0.010527,-0.075069,-0.058229,0.007484,-0.047799,-0.0

,assessment,assortment,catalog_navigation,delivery,payment,price,products_quality,promotions,support,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.089341,0.143146,-0.015899,-0.023591,-0.030676,0.091618,0.130106,-0.006714,0.149465,0.058533,0.119032,-0.017204,0.011735,0.063625,-0.136857,-0.104329,-0.040578,0.05209,0.179789,-0.026838,0.062827,0.006176,0.010871,0.01353,0.006867,-0.138088,0.116195,-0.008657,0.048361,-0.044996,0.146993,0.08961,-0.090399,0.098466,0.098074,0.047249,0.060935,0.095167,-0.160679,0.13872,0.15879,0.020652,0.077728,0.055672,-0.056016,-0.038539,0.053737,-0.016334,0.197879,0.059927,-0.002129,0.230212,-0.067982,0.016847,0.080604,-0.015574,-0.14584,0.030375,0.10569,-0.084736,-0.096943,-0.056284,-0.025466,0.030929,-0.047913,0.056936,0.040203,-0.168071,0.067153,-0.024962,0.103517,0.070333,-0.055894,-0.112288,-0.131552,0.070576,0.029893,-0.055033,-0.049267,0.042005,0.015632,-0.18164,-0.03711,-0.13731,-0.015124,-0.075044,-0.006796,0.014466,0.046141,0.071818,0.175275,-0.027512,0.022681,-0.115937,0.020336,-0.0121,0.005

,assessment,assortment,catalog_navigation,delivery,payment,price,products_quality,promotions,support,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,6.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.085804,0.148095,-0.020494,0.058895,0.009525,0.005931,0.12586,-0.087678,-0.043429,-0.0417,0.072719,0.01415,0.039616,0.00364,0.00693,-0.078012,-0.012843,0.060332,-0.001373,0.056089,0.0499,0.002155,0.032886,0.009165,0.0006,-0.029853,0.056189,0.021377,0.094874,0.056802,0.197383,-0.026826,0.009893,0.101852,0.087577,0.016345,0.030158,0.034717,-0.203183,0.038559,0.087224,-0.046659,0.016785,-0.041081,-0.013266,-0.045869,0.016224,0.01069,0.137624,0.038745,0.029136,0.209144,-0.107855,0.043828,0.019594,-0.145844,-0.066965,-0.035462,0.085821,-0.009389,-0.037259,-0.110228,-0.022449,-0.032332,-0.079204,0.008268,-0.049617,0.037444,-0.077637,-0.077637,0.040082,0.055955,-0.083195,0.053525,-0.06913,-0.042857,0.021592,-0.057744,-0.01649,0.017311,-0.036915,-0.015761,-0.052718,0.007496,-0.16791,0.060295,0.044048,0.028457,0.003044,0.066418,0.07559,-0.014686,0.013268,-0.070148,0.016633,-0.0867,0.125401,0.01

,assessment,assortment,catalog_navigation,delivery,payment,price,products_quality,promotions,support,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,5.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,-0.009093,-0.063334,-0.018352,0.005252,0.064992,0.023877,0.019063,0.032504,-0.035309,0.061967,0.06741,0.032137,0.016538,0.234396,-0.020323,-0.173322,0.031584,0.138215,-0.130143,0.011803,0.030621,-0.062919,0.037963,-0.106195,0.030514,-0.062881,0.0223,0.011327,-0.008152,0.100325,0.12446,0.029677,-0.006472,-0.082852,0.059181,-0.057136,-0.04136,0.033132,-0.268722,0.005782,0.094707,-0.050927,-0.034904,0.089426,-0.031168,0.080955,0.102185,0.02573,0.071287,0.040329,0.019156,0.042893,-0.009729,0.054874,0.079322,-0.188915,-0.08377,-0.094319,-0.009554,0.017099,0.080962,-0.04257,0.042072,-0.02353,-0.043256,0.00341,0.070171,-0.013851,-0.083196,-0.009639,0.168101,0.061676,-0.055074,0.062638,-0.063212,-0.246832,-0.086424,-0.000423,0.044801,0.057541,-0.014349,0.012343,-0.003106,0.051054,-0.04395,0.260917,0.033292,0.024742,-0.028277,-0.041386,0.078474,-0.090495,-0.017938,0.021615,0.110781,0.055139,0.08

## Предобучение классификатора.

Обучим классификатор на обучающей и провалидируем на валидационной подвыборках.<br>
Тестовую подвыборку отложим для проверки модели.

In [15]:
train_pool = Pool(X_train, y_train)
val_pool = Pool(X_val, y_val)
test_pool = Pool(X_test, y_test)

In [16]:
clf = CatBoostClassifier(
    loss_function='MultiLogloss',
    eval_metric='Accuracy',
    iterations=500,
    class_names=y_cols,
    random_state=SEED
)

clf.fit(train_pool, eval_set=val_pool, 
        metric_period=10, plot=False, verbose=50)

Learning rate set to 0.05508
0:	learn: 0.0469993	test: 0.0390033	best: 0.0390033 (0)	total: 4.12s	remaining: 34m 16s
50:	learn: 0.2089660	test: 0.1744312	best: 0.1744312 (50)	total: 3m 31s	remaining: 31m
100:	learn: 0.4815618	test: 0.2524377	best: 0.2524377 (100)	total: 6m 44s	remaining: 26m 37s
150:	learn: 0.7443962	test: 0.2806067	best: 0.2806067 (150)	total: 9m 55s	remaining: 22m 57s
200:	learn: 0.8919017	test: 0.2946912	best: 0.2946912 (200)	total: 13m 6s	remaining: 19m 30s
250:	learn: 0.9468547	test: 0.3185265	best: 0.3185265 (250)	total: 16m 13s	remaining: 16m 5s
300:	learn: 0.9790311	test: 0.3217768	best: 0.3217768 (260)	total: 19m 19s	remaining: 12m 46s
350:	learn: 0.9927693	test: 0.3261105	best: 0.3261105 (330)	total: 22m 29s	remaining: 9m 32s
400:	learn: 0.9960231	test: 0.3326111	best: 0.3326111 (390)	total: 25m 42s	remaining: 6m 20s
450:	learn: 0.9985539	test: 0.3304442	best: 0.3326111 (390)	total: 28m 54s	remaining: 3m 8s
499:	learn: 0.9989154	test: 0.3369447	best: 0.336944

## Самообучение классификатора.

Итеративно обучим классификатор на его же предсказаниях:
- классифицируем оставшиеся неразмеченные данные;
- добавим данные с предсказаниями с уверенностью выше заданной к размеченным данным;
- повторим итерацию.

### Функция самообучения.

In [17]:
def self_training(model : any, 
                  data_labeled : pd.DataFrame, 
                  data_unlabeled : pd.DataFrame, 
                  x_cols : list[str],
                  y_cols : list[str],
                  seed : int,
                  conf_th : float, 
                  iter : int):
    df = data_labeled.copy()
    X_self_train = data_unlabeled.copy()
    for i in range(iter):
        print(f'Самообучение. Шаг {i + 1}/{iter}')

        # Генерация индекса уверенных предсказаний.
        preds_proba = model.predict_proba(X_self_train)
        conf_idx = np.where(np.max(preds_proba, axis=1) > conf_th)[0]
        if len(conf_idx) == 0:
            print('Не нашлось уверенных предсказаний. Остановка.')
            break

        # Обновление датафреймов    
        conf_y = pd.DataFrame(model.predict(X_self_train)[conf_idx],  
                            index=conf_idx, columns=y_cols)
        conf_df = X_self_train.iloc[conf_idx].copy()
        conf_df = conf_df.join(conf_y, how='left')


        df = pd.concat([df, conf_df]).reset_index(drop=True)

        X_self_train = X_self_train.drop(X_self_train.index[conf_idx])
        X_self_train = X_self_train.reset_index(drop=True)

        # Повторное обучение на true и псевдоразметке 
        # (тестовая подвыборка не участвует в переобучении)
        x_train, x_eval, y_train, y_eval  = train_test_split(
            df[x_cols], df[y_cols], test_size=.2, random_state=seed
            )
        
        train_pool = Pool(x_train, y_train)
        val_pool = Pool(x_eval, y_eval)
        
        model.fit(X = train_pool, 
                  eval_set=val_pool, 
                  metric_period=10, 
                  plot=False, verbose=50)        

    return model

### Самообучение.

In [18]:
X_unlabeled.copy()
train_val_data = (pd
                  .concat([X_train, X_val])
                  .join(pd.concat([y_train, y_val]), how='left'))

self_training(
    model = clf, 
    data_labeled = train_val_data, 
    data_unlabeled = X_unlabeled, 
    x_cols = x_cols,
    y_cols = y_cols, 
    conf_th = .7, 
    iter = 3,
    seed = SEED
    )

Самообучение. Шаг 1/3
Learning rate set to 0.066643
0:	learn: 0.0987799	test: 0.1016043	best: 0.1016043 (0)	total: 4.06s	remaining: 33m 48s
50:	learn: 0.2774528	test: 0.2673797	best: 0.2673797 (50)	total: 3m 43s	remaining: 32m 46s
100:	learn: 0.3812469	test: 0.3288770	best: 0.3288770 (100)	total: 7m 22s	remaining: 29m 7s
150:	learn: 0.4698312	test: 0.3462567	best: 0.3462567 (150)	total: 11m 3s	remaining: 25m 32s
200:	learn: 0.5178004	test: 0.3529412	best: 0.3529412 (200)	total: 14m 39s	remaining: 21m 48s
250:	learn: 0.5565770	test: 0.3576203	best: 0.3596257 (220)	total: 18m 11s	remaining: 18m 2s
300:	learn: 0.5819823	test: 0.3562834	best: 0.3596257 (220)	total: 21m 40s	remaining: 14m 19s
350:	learn: 0.6035434	test: 0.3489305	best: 0.3596257 (220)	total: 25m 8s	remaining: 10m 40s
400:	learn: 0.6252716	test: 0.3469251	best: 0.3596257 (220)	total: 28m 40s	remaining: 7m 4s
450:	learn: 0.6373057	test: 0.3475936	best: 0.3596257 (220)	total: 32m 10s	remaining: 3m 29s
499:	learn: 0.6511783	tes

## Тестирование самообученной модели.

Протестируем модель на отложенной выборке.

In [21]:
test_predict = clf.predict(X_test)
print('Accuracy на отложенной выборке: '+
      f'{accuracy_score(y_test, test_predict)}')

Accuracy на отложенной выборке: 0.24702058504875407


## Выводы:

Самообучение с `CatBoostClassifier` не оправдало ожиданий.<br>
Возможно тестирование этого подхода с большой языковой моделью.